Implementation of Trie. Source: https://albertauyeung.github.io/2020/06/15/python-trie.html/

In [1]:
import pickle
import re
import uuid

In [2]:
class TrieNode(object):
    """
    Trie Node
    """
    
    def __init__(self, char):
    
        self.char = char
        self.is_end = False
        self.counter = 0
        self.children = {}

In [3]:
class Trie(object):
    """
    Trie object
    """
    
    def __init__(self):
        # head node is empty
        
        self.root = TrieNode("")
    
    def insert(self, word):
        """Insert a word into the trie"""
        node = self.root
        
        # Loop through each character in the word
        # Check if there is no child containing the character, create a new child for the current node
        for char in word:
            if char in node.children:
                node = node.children[char]
            else:
                # If a character is not found,
                # create a new node in the trie
                new_node = TrieNode(char)
                node.children[char] = new_node
                node = new_node
        
        # Mark the end of a word
        node.is_end = True

        # Increment the counter to indicate that we see this word once more
        node.counter += 1
    
    def dfs(self, node, prefix):
        """
        Depth-first search
        """
        
        if node.is_end:
            self.output.append((prefix + node.char, node.counter))
        
        for child in node.children.values():
            self.dfs(child, prefix + node.char)
    
    def query(self, x):
        """
        Given input (prefix), retrieve all words stored in a trie
        with that prefix. Sort the words by the number of times
        the word has been inserted
        """
        
        self.output = []
        node = self.root
        
        # check if prefix is in a trie
        for char in x:
            print(char)
            if char in node.children:
                node = node.children[char]
            else:
                return []
        
        self.dfs(node, x[:-1])
        
        return self.output

In [4]:
t = Trie()
t.insert("was")
t.insert("word")
t.insert("war")
t.insert("what")
t.insert("where")
t.insert("hello")
t.insert("high")
t.query("wh")
#[('what', 1), ('where', 1)]

w
h


[('what', 1), ('where', 1)]

**Problem Statement**:

We would like to implement search bar. We have a lot of lists of words (for example, news articles) and each of the list has random uuid. Our task is to find all instances of list id that would contain similar words. For example:

Assume we have 3 lists `list_1 = ['wall', 'ball', 'hello']`, `list_2 = ['wall', 'well', 'name']`, `list_3 = ['well', 'smoke', 'wood']`.
If we input `wa` we want to get id of list_1 and list_2.

**How to do it?**

First, we construct input dictionary: `{word1 = [ids,,], word2: [ids...]}` where keys are unique words. Given that, we could use `for` loop and find appropriate IDs. However, this is slow and inneficient. Other way is to use Trie!

**Prepare Data**

In [5]:
with open('test_data.pickle', 'rb') as f:
    test_data = pickle.load(f)

In [6]:
test_data_no_numbers = list(
    map(lambda x: re.sub('\d', '', x), test_data)
)

test_data_no_numbers = list(
    map(lambda x: list(set(x.lower().split(';'))), test_data_no_numbers)
)

In [7]:
def list_of_lists_to_set(input_data: list) -> set:
    """
    Given input list of lists of strings,
    cast it to set with unique words only
    """
    
    
    temp = []
    
    for i in input_data:
        temp += i.split(' ')
    return list(set(temp))

In [8]:
test_data_no_numbers = list(
    map(lambda x: list_of_lists_to_set(x), test_data_no_numbers)
)

**Create Common Dictionary**

We want to create the following data structure:

```
{
    'word1': [uuid1, uuid2...],
    'word2': [uuid1, uuid2...],
}
```

In this data structure, we have unique words from all documents as keys
and values are IDs of documents where they appear. Technically we could compute
all possible combinations and then look that up which would give us O(1) time complexity but O(number of possible combinations) space complexity

Given this structure, our search bar would work in the following way:

User starts typing word (like "hello", we find all the articles that has word "hello" and show them)

In [9]:
# create intermediary dictionary:
# {'id': [words], ...}

intermediate = dict(zip([str(uuid.uuid4()) for _ in range(len(test_data_no_numbers))], test_data_no_numbers))

In [10]:
# create set of all words from the articles
all_words = []

for a in test_data_no_numbers:
    all_words += a
all_words = set(all_words)

In [11]:
# given all_words set, construct dictionary:
# {word: [id1, id2],..}

words_to_ids = {}

for w in all_words:
    ids = []
    for k, v in intermediate.items():
        if w in v:
            ids.append(k)
    words_to_ids[w] = ids

In [18]:
def test_global_dictionary(
    words_to_ids: dict,
    ids_to_words: dict,
    words: list
):
    """
    Test if words to ids dictionary
    is correctly done by providing list
    of test words
    """
    
    for w in words:
        ids_to_test = words_to_ids[w]
        for id_ in ids_to_test:
            if w not in ids_to_words[id_]:
                raise ValueError(f"Word {w} not found!")
            else:
                print(ids_to_words[id_])

In [19]:
test_global_dictionary(words_to_ids, intermediate, words=['bath', 'tom', 'helicopter'])

['twitter,', 'jewish', 'facebook,', 'beyond,', 'bed', 'museum,', 'bath']
['iron', 'bath', 'works,']
['city', 'bath', 'council,']
['york', 'facebook,', 'oxford', 'school', 'bath', 'university,', 'of', 'medicine,', 'university']
['iron', 'us', 'press,', 'works,', 'naval', 'bath', 'navy,', 'academy,', 'associated']
['elemental', 'joy', 'facial', 'herbology', 'cleanse', 'oil,', 'bath', 'beatitude', 'cleansing', 'harmonising']
['malaysia,', 'in', 'racquets', 'mayfair,', 'club', 'high', 'national', 'tree,', 'opera,', 'group', 'bath', 'of', 'court,', 'lion', 'english']
['cross', 'church,', 'mackintosh', 'museum', 'at', 'albert', 'museum,', 'bath', 'of', 'glasgow,', 'queen', 'work,', 'university']
['twitter,', 'neiman', 'facebook,', 'beyond,', 'soho', 'in', 'nordstrom,', 'bed', 'bag', 'diaper', 'marcus,', 'bath', 'solutions', 'dahlia,']
['twitter,', 'deputy', 'tom', 'leader', 'oregon', 'state', 'university,', 'watson,', 'party', 'labour']
['research', 'parnell,', 'development', 'tom']
['church

In [20]:
class TrieNode(object):
    """
    Trie Node
    """
    
    def __init__(self, char):
    
        self.char = char
        self.is_end = False
        self.ids = []
        self.children = {}

In [21]:
class Trie(object):
    """
    Trie object
    """
    
    def __init__(self):
        # head node is empty
        
        self.root = TrieNode("")
    
    def insert(self, word: str, ids: list):
        """Insert a word into the trie"""
        node = self.root
        
        # Loop through each character in the word
        # Check if there is no child containing the character, create a new child for the current node
        for char in word:
            if char in node.children:
                node = node.children[char]
                node.ids += ids
            else:
                # If a character is not found,
                # create a new node in the trie
                new_node = TrieNode(char)
                node.children[char] = new_node
                node = new_node
                node.ids += ids
        
        # Mark the end of a word
        node.is_end = True
    
    def dfs(self, node, prefix):
        """
        Depth-first search
        """
        
        if node.is_end:
            self.output.append((prefix + node.char, node.ids))
        
        for child in node.children.values():
            self.dfs(child, prefix + node.char)
    
    def query(self, x):
        """
        Given input (prefix), retrieve all words stored in a trie
        with that prefix. Sort the words by the number of times
        the word has been inserted
        """
        
        self.output = []
        node = self.root
        
        # check if prefix is in a trie
        for char in x:
            if char in node.children:
                node = node.children[char]
            else:
                return []
        
        self.dfs(node, x[:-1])
        
        return self.output

In [23]:
t = Trie()

for k, v in words_to_ids.items():
    t.insert(k, v)

In [24]:
# test alternate implementation
query_result = []
for k, v in words_to_ids.items():
    if 'tom' in k:
        query_result += v

In [25]:
t.query('tom')

[('tom',
  ['e6efe78c-93f8-493a-861b-06a658f98a44',
   '742b7772-26a0-4b54-8d7e-c2e302a4120c',
   '76f5f33a-4f4e-4782-8d0d-acfce03b4fc1',
   '04555399-a56a-4a39-8c3f-96e170774322',
   '0eeefb49-6d4a-4b1e-9dd2-8bf5c0c0ef48',
   '4f07b53c-7fa2-4412-8303-447efd261795',
   'eea6bf6b-3dc6-4f90-8c50-7399d78ce024',
   '29dd6fb7-668c-4bf9-8a9f-2fd63e5e2a45',
   'bc899bf2-584e-488d-b16a-84329cd01e57',
   '6212cced-d4a7-404a-8f28-990ba67b3a03',
   'd37753d6-4987-45de-bf34-1133f5fbc0fa',
   '7336cc4f-f308-45c0-b595-376f52e6207d',
   'f33a34ea-53b6-48cb-820f-118c4b2e3460',
   'facc2af4-7749-4c53-881c-8f3156c13bbb',
   '370dfb02-089d-46a6-a8f0-89cebedca173',
   'f7f7c2b7-5680-4665-b6b1-76b8aa68144d',
   '3820403d-6f33-476f-b2cc-ed2de8683829',
   'c5aa7075-139d-41af-a9c9-16392e7e9b12',
   'ffcb2c3c-1c6a-422b-a47d-de96d79c5541',
   '5822d848-5da0-4e32-9dcd-be87e5fb7c62',
   'e550ca48-ea83-4429-8253-dcebf99ef021',
   '083cabfb-3ee7-49f0-a7e9-5716d2897c06',
   '3c7ef838-947c-4250-bd67-9181a2d9e0fb',
  